In [1]:
# Path hack so that we can import see library.
import sys, os
sys.path.insert(0, os.path.abspath('..'))

## Defining you own parameter space
This notebook demonstrates how one may define their own Classifier Search Space!

In [2]:
from see.classifiers import Classifier, ClassifierParams

We will use the `Classifier#set_search_space` class method to set our custom search space to be the Classifier Search Space that we can run Genetic Search over. We first define a simple custom space dictionaries that will be passed into `Classifier#set_search_space`.

In [3]:
# First, initialize custom search space dictionaries.

# These two dictionaries represent
# the algorithm space and the
# parameter space respectively.
# The keys of these dictionaries
# are the algorithm names and
# the parameter names respectively.
new_algo_space = dict()
new_param_space = dict()

# The values of the new_algo_space
# dictionary needs to be ClassifierContainer
# Classes (not instances!). Many have already
# been pre-defined
# in the classifiers.py file.

# The values of the parameter space dictionary (i.e. new_param_space)
# needs to be a tuple: (range, description).
# `range` is a list of possible values for the
# the corresponding parameter name.
# `description` is used to document and
# describe the parameter.

In [4]:
# To add a Classifier algorithm to the 
# search space, one first needs to pick or define
# the corresponding classifier container.
# Existing Classifier Containers have
# been defined in classifiers.py.

# First add import and add the container 
# to the algorithm space dictionary.
from see.classifiers import MLPContainer

new_algo_space["MLP Neural Network"] = MLPContainer

# Then, add its parameters to the parameter space dictionary
for param in MLPContainer().paramindexes:
    new_param_space[param] = ClassifierParams.get_param(param)
    
# Right now in order to add a new classifier to the search space
# one would have to define a ClassifierContainer class
# and add its respective parameters and parameter ranges carefully.
# There is not a simpler method 
# to directly add Classifier and custom search space yet.

In [5]:
# Alternatively, one may manually define a search space within
# the Classifier class itself. Look at the Classifier#use_dhahri_space
# and Classifier#use_tutorial_space class methods as an example.

In [6]:
# To set the custom search space as the Classifier space, run:
Classifier.set_search_space(new_algo_space, new_param_space)

In [7]:
# Check the search space by running:
# Initialize Algorithm Space and Workflow
import pprint

algorithm_space = Classifier.algorithmspace
pp = pprint.PrettyPrinter(indent=4)
pp.pprint(algorithm_space)

# Print possible parameters
print('Possible Parameters')
pp.pprint(ClassifierParams.pkeys)
print('\n')
# Print parameter ranges
print('Parameter Ranges')
pp.pprint(ClassifierParams.ranges)
print('\n')
print('Parameter Descriptions')
# Print descriptions of parameters
pp.pprint(ClassifierParams.descriptions)

{'MLP Neural Network': <class 'see.classifiers.MLPContainer'>}
Possible Parameters
['algorithm', 'activation', 'alpha', 'max_iter', 'solver']


Parameter Ranges
{   'activation': ['identity', 'logistic', 'tanh', 'relu'],
    'algorithm': ['MLP Neural Network'],
    'alpha': [1e-06, 1e-05, 0.0001, 0.001, 0.01, 0.1],
    'max_iter': [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000],
    'solver': ['lbfgs', 'sgd', 'adam']}


Parameter Descriptions
{   'activation': 'Activation function for the hidden layer of neural network.',
    'algorithm': 'string code for the algorithm',
    'alpha': 'L2 penalty regularization parameter',
    'max_iter': 'Number of iterations for the algorithm',
    'solver': 'The solver for weight optimization of neural network.'}


Let's test our custom workspace using Genetic Search!

In [8]:
from see.Workflow import workflow
from see.classifier_fitness import ClassifierFitness

# Initialize workflow for Genetic Search
workflow.addalgos([Classifier, ClassifierFitness])
wf = workflow()
print(wf)

<class 'see.Workflow.workflow'> parameters: 
	algorithm = MLP Neural Network
	activation = identity
	alpha = 1e-06
	max_iter = 100
	solver = lbfgs



In [9]:
%%time
from see import GeneticSearch
num_generations = 15

from see.base_classes import pipedata
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_circles
from see.classifier_helpers import helpers

# Create Simple Dataset
circles_ds = pipedata()
circles_ds.name = 'Circles Dataset'
circles_ds.X, circles_ds.y = make_circles(noise=0.2, factor=0.5, random_state=1)

# Preprocess data
circles_ds.X = StandardScaler().fit_transform(circles_ds.X)

# Create pipeline data that can be passed into the classifier pipeline
pipeline_data = helpers.generate_train_test_set(circles_ds.X, circles_ds.y)

my_evolver = GeneticSearch.Evolver(workflow, pipeline_data, pop_size=15)
my_evolver.run(ngen=num_generations, print_raw_data=True)

Initializing a new random population
Generation 0/15 of population size 15
<class 'see.Workflow.workflow'> parameters: 
	algorithm = MLP Neural Network
	activation = tanh
	alpha = 0.01
	max_iter = 800
	solver = adam



/home/davin/Documents/GitHub/see-segment/see-clasify/envs/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (800) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/davin/Documents/GitHub/see-segment/see-clasify/envs/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(


Time: 0.671 s
fitness=0.125

<class 'see.Workflow.workflow'> parameters: 
	algorithm = MLP Neural Network
	activation = relu
	alpha = 0.01
	max_iter = 100
	solver = sgd

Time: 0.077 s
fitness=0.55

<class 'see.Workflow.workflow'> parameters: 
	algorithm = MLP Neural Network
	activation = relu
	alpha = 0.0001
	max_iter = 800
	solver = adam



/home/davin/Documents/GitHub/see-segment/see-clasify/envs/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (800) reached and the optimization hasn't converged yet.
  warnings.warn(


Time: 0.611 s
fitness=0.125

<class 'see.Workflow.workflow'> parameters: 
	algorithm = MLP Neural Network
	activation = tanh
	alpha = 0.01
	max_iter = 100
	solver = adam

Time: 0.03 s
fitness=0.6

<class 'see.Workflow.workflow'> parameters: 
	algorithm = MLP Neural Network
	activation = identity
	alpha = 1e-06
	max_iter = 400
	solver = lbfgs

Time: 0.006 s
fitness=0.6

<class 'see.Workflow.workflow'> parameters: 
	algorithm = MLP Neural Network
	activation = logistic
	alpha = 0.1
	max_iter = 300
	solver = adam

Time: 0.021 s
fitness=0.6

<class 'see.Workflow.workflow'> parameters: 
	algorithm = MLP Neural Network
	activation = logistic
	alpha = 1e-06
	max_iter = 500
	solver = sgd

Time: 0.01 s
fitness=0.6

<class 'see.Workflow.workflow'> parameters: 
	algorithm = MLP Neural Network
	activation = identity
	alpha = 0.0001
	max_iter = 700
	solver = adam

Time: 0.027 s
fitness=0.6

<class 'see.Workflow.workflow'> parameters: 
	algorithm = MLP Neural Network
	activation = logistic
	alpha = 

...

Time: 0.251 s
fitness=0.1

<class 'see.Workflow.workflow'> parameters: 
	algorithm = MLP Neural Network
	activation = relu
	alpha = 0.01
	max_iter = 300
	solver = adam

Time: 0.19 s
fitness=0.1

<class 'see.Workflow.workflow'> parameters: 
	algorithm = MLP Neural Network
	activation = relu
	alpha = 0.001
	max_iter = 600
	solver = adam



/home/davin/Documents/GitHub/see-segment/see-clasify/envs/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (600) reached and the optimization hasn't converged yet.
  warnings.warn(


Time: 0.384 s
fitness=0.125

<class 'see.Workflow.workflow'> parameters: 
	algorithm = MLP Neural Network
	activation = relu
	alpha = 0.001
	max_iter = 500
	solver = adam

Time: 0.32 s
fitness=0.1

#BEST [0.1,  ['MLP Neural Network', 'relu', 0.0001, 400, 'adam']]
# GEN HOF_index fitness ind|15;0;0.1;['MLP Neural Network', 'relu', 0.0001, 400, 'adam']
# GEN HOF_index fitness ind|15;1;0.1;['MLP Neural Network', 'relu', 0.01, 300, 'adam']
# GEN HOF_index fitness ind|15;2;0.1;['MLP Neural Network', 'relu', 0.0001, 300, 'adam']
# GEN HOF_index fitness ind|15;3;0.1;['MLP Neural Network', 'relu', 0.01, 400, 'adam']
# GEN HOF_index fitness ind|15;4;0.1;['MLP Neural Network', 'relu', 0.001, 600, 'adam']
# GEN HOF_index fitness ind|15;5;0.1;['MLP Neural Network', 'relu', 0.001, 500, 'adam']
# GEN HOF_index fitness ind|15;6;0.1;['MLP Neural Network', 'relu', 0.001, 400, 'adam']
# GEN HOF_index fitness ind|15;7;0.125;['MLP Neural Network', 'relu', 0.001, 100, 'lbfgs']
# GEN HOF_index fitness ind|1

/home/davin/Documents/GitHub/see-segment/see-clasify/envs/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


[['MLP Neural Network', 'tanh', 1e-06, 900, 'sgd'],
 ['MLP Neural Network', 'relu', 0.0001, 700, 'sgd'],
 ['MLP Neural Network', 'logistic', 1e-06, 800, 'sgd'],
 ['MLP Neural Network', 'relu', 0.0001, 600, 'sgd'],
 ['MLP Neural Network', 'relu', 1e-05, 400, 'lbfgs'],
 ['MLP Neural Network', 'logistic', 0.001, 700, 'lbfgs'],
 ['MLP Neural Network', 'tanh', 0.0001, 300, 'adam'],
 ['MLP Neural Network', 'relu', 0.0001, 400, 'adam'],
 ['MLP Neural Network', 'relu', 0.01, 300, 'adam'],
 ['MLP Neural Network', 'relu', 0.0001, 400, 'adam'],
 ['MLP Neural Network', 'relu', 0.01, 300, 'adam'],
 ['MLP Neural Network', 'relu', 0.0001, 300, 'adam'],
 ['MLP Neural Network', 'relu', 0.01, 400, 'adam'],
 ['MLP Neural Network', 'relu', 0.001, 600, 'adam'],
 ['MLP Neural Network', 'relu', 0.001, 500, 'adam']]